In [1]:
import numpy as np
import pandas as pd

from pyquaternion import Quaternion

from trackml.dataset import load_event, load_dataset
from trackml.score import score_event

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

from sklearn.cluster import KMeans, DBSCAN
from scipy.spatial.distance import cdist
from scipy.sparse.csgraph import connected_components
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

%matplotlib inline

In [2]:
R=2000 #Helix Radius
w=np.pi/3 #length of arc
v=3000 # velocity in Z
theta0=-0.3 # angle at the origin
D=0 #D=500 - Use for helixes far from the origin
z0=0 #z0=200 - Z shifting
t=np.random.rand(50)
X=-R*np.sin(w*t+theta0)+(R-D)*np.sin(theta0)
Y=-R*np.cos(w*t+theta0)+(R-D)*np.cos(theta0)
Z=z0+v*t

In [41]:
df = pd.DataFrame()
df['x'] = -R*np.sin(w*t+theta0)+(R-D)*np.sin(theta0)
df['y'] = -R*np.cos(w*t+theta0)+(R-D)*np.cos(theta0)
df['z']=z0+v*t

In [42]:
df.head()

,x,y,z
0,-1297.610236,39.641422,1893.855048
1,-785.855777,-79.816151,1138.932328
2,-1289.911092,36.752016,1882.075785
3,-1351.342352,60.824076,1976.588411
4,-1320.098633,48.288992,1928.367003


In [43]:
x = df['x'].values
y = df['y'].values
z = df['z'].values
r = np.sqrt(x**2 + y**2 + z**2)
rt = np.sqrt(x**2 + y**2 )
df['zrt'] = z/np.sqrt(x**2 + y**2)
df['zr'] = z/np.sqrt(x**2 + y**2 + z**2)
zrt = df.zrt.values
zr = df.zr.values
m_lambda = zrt
m_lambda2 = zr
df['z_rt'] = np.sqrt(1 + (m_lambda**2))
df['z_r'] = np.sqrt(1 + (m_lambda2**2))

df['z_rt_rt'] = np.sqrt(1 + (m_lambda**2))/rt
df['z_r_r'] = np.sqrt(1 + (m_lambda2**2))/r

In [44]:
df.head(50)

,x,y,z,zrt,zr,z_rt,z_r,z_rt_rt,z_r_r
0,-1297.610236,39.641422,1893.855048,1.458814,0.824816,1.768654,1.296272,0.001362,0.000565
1,-785.855777,-79.816151,1138.932328,1.441871,0.821717,1.754706,1.294302,0.002221,0.000934
2,-1289.911092,36.752016,1882.075785,1.458482,0.824756,1.768381,1.296234,0.001370,0.000568
3,-1351.342352,60.824076,1976.588411,1.461206,0.825247,1.770628,1.296547,0.001309,0.000541
4,-1320.098633,48.288992,1928.367003,1.459799,0.824994,1.769467,1.296385,0.001340,0.000555
5,-348.069987,-74.513505,510.545382,1.434292,0.820306,1.748483,1.293407,0.004912,0.002078
6,-1335.570718,54.420133,1952.205848,1.460490,0.825118,1.770037,1.296464,0.001324,0.000548
7,-306.958042,-69.048519,451.137787,1.433876,0.820229,1.748142,1.293358,0.005556,0.002351
8,-153.899567,-40.969376,228.182646,1.432773,0.820022,1.747238,1.293227,0.010971,0.004647
9,-26.904608,-8.115909,40.253570,1.432406,0.819953,1.746937,1.293183,0.062164,0.026342


In [20]:
# def run_dbscan():

data_dir = '../data/train'

#     event_ids = [
#             '000001030',##
#             '000001025','000001026','000001027','000001028','000001029',
#     ]

event_ids = [
        '000001030',##

]

sum=0
sum_score=0
for i,event_id in enumerate(event_ids):
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    hits  = pd.read_csv(data_dir + '/event%s-hits.csv'%event_id)
    cells = pd.read_csv(data_dir + '/event%s-cells.csv'%event_id)
    truth = pd.read_csv(data_dir + '/event%s-truth.csv'%event_id)
    particles = pd.read_csv(data_dir + '/event%s-particles.csv'%event_id)
    
    truth = pd.merge(truth, particles, how='left', on='particle_id')
    hits = pd.merge(hits, truth, how='left', on='hit_id')

In [21]:
x = hits['x'].values
y = hits['y'].values
z = hits['z'].values

hits['rt'] = np.sqrt(x**2 + y**2)
rt = hits['rt'].values

hits['r'] = np.sqrt(x**2 + y**2 + z**2)
r = hits['r'].values

hits['m_z_rt_r'] = np.sqrt(1 + ((z/rt)**2))/r
hits['m_z_rt_rt'] = np.sqrt(1 + ((z/rt)**2))/rt
hits['m_z_r_r'] = np.sqrt(1 + ((z/r)**2))/r
hits['m_z_rt'] = np.sqrt(1 + ((z/rt)**2))
hits['m_z_r'] = np.sqrt(1 + ((z/r)**2))
hits['m_z_rt'] = np.sqrt(1 + ((z/rt)**2))

In [22]:
hits.head()

,hit_id,x,y,z,volume_id,layer_id,module_id,particle_id,tx,ty,tz,tpx,tpy,tpz,weight,vx,vy,vz,px,py,pz,q,nhits,rt,r,m_z_rt_r,m_z_rt_rt,m_z_r_r,m_z_rt,m_z_r
0,1,-60.8722,-2.90994,-1502.5,7,2,1,427858663433043968,-60.8900,-2.93122,-1502.5,-0.389865,-0.039903,-9.71788,0.000008,0.010578,-0.000096,-2.68331,-0.396593,-0.000155,-9.72165,-1.0,12.0,60.941714,1503.735396,0.016409,0.404895,0.000940,24.674977,1.413633
1,2,-68.6209,-6.93030,-1502.5,7,2,1,923241222145835008,-68.6150,-6.94011,-1502.5,-0.225243,-0.003877,-4.86031,0.000008,0.012087,-0.023069,13.10000,-0.216069,-0.041984,-4.86522,1.0,13.0,68.969972,1504.082148,0.014499,0.316192,0.000940,21.807783,1.413470
2,3,-64.5802,2.66632,-1502.5,7,2,1,4523734434054144,-64.5634,2.67475,-1502.5,-0.284755,-0.008789,-6.74716,0.000006,-0.009604,0.008012,-2.28776,-0.292098,0.031685,-6.75259,-1.0,16.0,64.635219,1503.889611,0.015471,0.359979,0.000940,23.267340,1.413560
3,4,-88.7343,-7.35970,-1502.5,7,2,1,261225408500858880,-88.7490,-7.38212,-1502.5,-0.904795,-0.049219,-15.14960,0.000012,0.001303,0.017286,-8.23350,-0.894095,-0.099110,-15.15630,1.0,14.0,89.038987,1505.135938,0.011231,0.189852,0.000939,16.904235,1.412976
4,5,-88.3262,-13.04520,-1502.5,7,2,1,743099023757410304,-88.3163,-13.05070,-1502.5,-0.295502,-0.016784,-4.90527,0.000006,-0.011713,-0.011290,-5.07843,-0.283424,-0.068196,-4.91150,1.0,16.0,89.284348,1505.150472,0.011200,0.188812,0.000939,16.857943,1.412969


In [23]:
hits1 = hits.loc[hits.particle_id == 427858663433043968]

In [24]:
hits1

,hit_id,x,y,z,volume_id,layer_id,module_id,particle_id,tx,ty,tz,tpx,tpy,tpz,weight,vx,vy,vz,px,py,pz,q,nhits,rt,r,m_z_rt_r,m_z_rt_rt,m_z_r_r,m_z_rt,m_z_r
0,1,-60.8722,-2.909940,-1502.5,7,2,1,427858663433043968,-60.8900,-2.931220,-1502.5,-0.389865,-0.039903,-9.71788,0.000008,0.010578,-0.000096,-2.68331,-0.396593,-0.000155,-9.72165,-1.0,12.0,60.941714,1503.735396,0.016409,0.404895,0.000940,24.674977,1.413633
1343,1344,-60.7069,-2.920990,-1497.5,7,2,106,427858663433043968,-60.6894,-2.911450,-1497.5,-0.390308,-0.036831,-9.71844,0.000007,0.010578,-0.000096,-2.68331,-0.396593,-0.000155,-9.72165,-1.0,12.0,60.777133,1498.732835,0.016454,0.405736,0.000943,24.659486,1.413632
1376,1377,-52.8537,-2.228720,-1302.5,7,4,1,427858663433043968,-52.8376,-2.231830,-1302.5,-0.392057,-0.031362,-9.71843,0.000006,0.010578,-0.000096,-2.68331,-0.396593,-0.000155,-9.72165,-1.0,12.0,52.900669,1303.573830,0.018903,0.465815,0.001084,24.641916,1.413631
3004,3005,-52.6381,-2.201850,-1297.5,7,4,106,427858663433043968,-52.6359,-2.215200,-1297.5,-0.392496,-0.033162,-9.71943,0.000004,0.010578,-0.000096,-2.68331,-0.396593,-0.000155,-9.72165,-1.0,12.0,52.684132,1298.569162,0.018981,0.467849,0.001089,24.648203,1.413631
3052,3053,-44.7498,-1.590930,-1102.5,7,6,1,427858663433043968,-44.7495,-1.607330,-1102.5,-0.393686,-0.027787,-9.71944,0.000004,0.010578,-0.000096,-2.68331,-0.396593,-0.000155,-9.72165,-1.0,12.0,44.778071,1103.408957,0.022332,0.550308,0.001281,24.641726,1.413631
5014,5015,-44.5357,-1.585650,-1097.5,7,6,106,427858663433043968,-44.5469,-1.592620,-1097.5,-0.393998,-0.029333,-9.72027,0.000004,0.010578,-0.000096,-2.68331,-0.396593,-0.000155,-9.72165,-1.0,12.0,44.563919,1098.404385,0.022440,0.553090,0.001287,24.647841,1.413631
5059,5060,-39.0549,-1.192890,-962.5,7,8,1,427858663433043968,-39.0603,-1.215860,-962.5,-0.395394,-0.025388,-9.72026,0.000005,0.010578,-0.000096,-2.68331,-0.396593,-0.000155,-9.72165,-1.0,12.0,39.073114,963.292769,0.025593,0.630961,0.001467,24.653596,1.413632
5097,5098,-38.8700,-1.181670,-958.0,7,8,4,427858663433043968,-38.8778,-1.204450,-958.0,-0.394204,-0.023836,-9.72089,0.000007,0.010578,-0.000096,-2.68331,-0.396593,-0.000155,-9.72165,-1.0,12.0,38.887958,958.788962,0.025715,0.634005,0.001474,24.655164,1.413632
7336,7337,-38.8468,-1.180760,-957.5,7,8,106,427858663433043968,-38.8574,-1.203210,-957.5,-0.394831,-0.024185,-9.72090,0.000008,0.010578,-0.000096,-2.68331,-0.396593,-0.000155,-9.72165,-1.0,12.0,38.864741,958.288432,0.025730,0.634431,0.001475,24.657013,1.413632
7362,7363,-33.3794,-0.896683,-822.5,7,10,1,427858663433043968,-33.3703,-0.891793,-822.5,-0.395397,-0.020699,-9.72093,0.000010,0.010578,-0.000096,-2.68331,-0.396593,-0.000155,-9.72165,-1.0,12.0,33.391442,823.177525,0.029948,0.738283,0.001717,24.652350,1.413632


In [25]:
hits2 = hits.loc[hits.particle_id == 923241222145835008]

In [26]:
hits2

,hit_id,x,y,z,volume_id,layer_id,module_id,particle_id,tx,ty,tz,tpx,tpy,tpz,weight,vx,vy,vz,px,py,pz,q,nhits,rt,r,m_z_rt_r,m_z_rt_rt,m_z_r_r,m_z_rt,m_z_r
1,2,-68.6209,-6.93030,-1502.5,7,2,1,923241222145835008,-68.6150,-6.94011,-1502.5,-0.225243,-0.003877,-4.86031,0.000008,0.012087,-0.023069,13.1,-0.216069,-0.041984,-4.86522,1.0,13.0,68.969972,1504.082148,0.014499,0.316192,0.000940,21.807783,1.413470
1398,1399,-59.3290,-6.70056,-1302.5,7,4,1,923241222145835008,-59.3259,-6.68824,-1302.5,-0.225150,-0.009007,-4.86120,0.000007,0.012087,-0.023069,13.1,-0.216069,-0.041984,-4.86522,1.0,13.0,59.706178,1303.867738,0.016749,0.365759,0.001084,21.838071,1.413472
1432,1433,-59.1315,-6.66307,-1298.0,7,4,4,923241222145835008,-59.1179,-6.68030,-1298.0,-0.224315,-0.008262,-4.86210,0.000006,0.012087,-0.023069,13.1,-0.216069,-0.041984,-4.86522,1.0,13.0,59.505721,1299.363279,0.016805,0.366955,0.001088,21.835939,1.413472
3043,3044,-50.1129,-6.22600,-1102.5,7,6,1,923241222145835008,-50.1160,-6.24249,-1102.5,-0.223324,-0.013702,-4.86217,0.000004,0.012087,-0.023069,13.1,-0.216069,-0.041984,-4.86522,1.0,13.0,50.498176,1103.655887,0.019803,0.432795,0.001281,21.855361,1.413473
3109,3110,-49.9169,-6.25058,-1098.0,7,6,4,923241222145835008,-49.9106,-6.23100,-1098.0,-0.220650,-0.012124,-4.86291,0.000004,0.012087,-0.023069,13.1,-0.216069,-0.041984,-4.86522,1.0,13.0,50.306726,1099.151840,0.019878,0.434316,0.001286,21.849004,1.413473
5066,5067,-43.7684,-5.82699,-962.5,7,8,1,923241222145835008,-43.7724,-5.84146,-962.5,-0.219941,-0.015901,-4.86297,0.000004,0.012087,-0.023069,13.1,-0.216069,-0.041984,-4.86522,1.0,13.0,44.154577,963.512261,0.022648,0.494203,0.001467,21.821345,1.413471
5118,5119,-43.5726,-5.80729,-958.0,7,8,4,923241222145835008,-43.5687,-5.82613,-958.0,-0.220586,-0.016711,-4.86380,0.000004,0.012087,-0.023069,13.1,-0.216069,-0.041984,-4.86522,1.0,13.0,43.957890,959.007975,0.022749,0.496305,0.001474,21.816515,1.413471
7387,7388,-37.4356,-5.27844,-822.5,7,10,1,923241222145835008,-37.4252,-5.29168,-822.5,-0.220169,-0.021097,-4.86384,0.000006,0.012087,-0.023069,13.1,-0.216069,-0.041984,-4.86522,1.0,13.0,37.805900,823.368408,0.026451,0.576070,0.001717,21.778834,1.413468
7457,7458,-37.2232,-5.25991,-818.0,7,10,4,923241222145835008,-37.2222,-5.27220,-818.0,-0.219298,-0.021314,-4.86436,0.000007,0.012087,-0.023069,13.1,-0.216069,-0.041984,-4.86522,1.0,13.0,37.592995,818.863379,0.026601,0.579425,0.001726,21.782339,1.413468
9978,9979,-37.1940,-5.27305,-817.5,7,10,106,923241222145835008,-37.1997,-5.27005,-817.5,-0.219050,-0.020910,-4.86442,0.000008,0.012087,-0.023069,13.1,-0.216069,-0.041984,-4.86522,1.0,13.0,37.565925,818.362663,0.026620,0.579906,0.001727,21.784707,1.413468
